In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras import Sequential
from keras import layers
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout
from keras.optimizers import adam_v2
from tensorflow.keras.utils import to_categorical
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
import random
from tqdm.notebook import tqdm
import collections

In [3]:
train_data = np.array(pd.read_pickle('F:/df/NoDef/X_train_NoDef.pkl')) # shape (76000, 5000), 95 website, 800 repeat
train_index = np.array(pd.read_pickle('F:/df/NoDef/y_train_NoDef.pkl')) # shape (76000,), 0-94 range, random

test_data = np.array(pd.read_pickle('F:/df/NoDef/X_test_NoDef.pkl')) # shape (9500, 5000), 95 website, 100 repeat
test_index = np.array(pd.read_pickle('F:/df/NoDef/y_test_NoDef.pkl')) # shape (9500,), 0-94 range, sequence

In [4]:
train_data = train_data[0 : 7600]
train_index = train_index[0 : 7600]

In [5]:
def get_data_length(data_vector):
    data_length = 5000
    for i in range(5000):
        if data_vector[i] == 0:
            data_length = i
            break
    return data_length

def add_noise(data_vector, data_length, count):
    for i in range(count):
        location = random.randint(0, data_length - 1)
        packet = random.randint(0, 1) * 2 - 1
        data_vector = np.insert(data_vector, location, packet)
        if data_length < 5000:
            data_length += 1
    
    return data_vector[0 : 5000]

In [8]:
train_data_aug = train_data.copy()
train_index_aug = train_index.copy()
for j in range(5):
    train_data_aug_one = train_data.copy()
    for i in tqdm(range(7600)):
        length = get_data_length(train_data_aug_one[i])
        rate = random.randint(0, 100) / 100
        train_data_aug_one[i] = add_noise(train_data_aug_one[i], length, int(length * rate))
    train_data_aug = np.append(train_data_aug, train_data_aug_one, axis=0)
    train_index_aug = np.append(train_index_aug, train_index, axis=0)

  0%|          | 0/7600 [00:00<?, ?it/s]

  0%|          | 0/7600 [00:00<?, ?it/s]

  0%|          | 0/7600 [00:00<?, ?it/s]

  0%|          | 0/7600 [00:00<?, ?it/s]

  0%|          | 0/7600 [00:00<?, ?it/s]

In [9]:
# DF model used for non-defended dataset
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, BatchNormalization
from keras.layers.core import Activation, Flatten, Dense, Dropout
from keras.layers.activation import ELU
from keras.initializers import glorot_uniform

class DFNet:
    @staticmethod
    def build(input_shape, classes):
        model = Sequential()
        #Block1
        filter_num = ['None',32,64,128,256]
        kernel_size = ['None',8,8,8,8]
        conv_stride_size = ['None',1,1,1,1]
        pool_stride_size = ['None',4,4,4,4]
        pool_size = ['None',8,8,8,8]

        model.add(Conv1D(filters=filter_num[1], kernel_size=kernel_size[1], input_shape=input_shape,
                         strides=conv_stride_size[1], padding='same',
                         name='block1_conv1'))
        model.add(BatchNormalization(axis=-1))
        model.add(ELU(alpha=1.0, name='block1_adv_act1'))
        model.add(Conv1D(filters=filter_num[1], kernel_size=kernel_size[1],
                         strides=conv_stride_size[1], padding='same',
                         name='block1_conv2'))
        model.add(BatchNormalization(axis=-1))
        model.add(ELU(alpha=1.0, name='block1_adv_act2'))
        model.add(MaxPooling1D(pool_size=pool_size[1], strides=pool_stride_size[1],
                               padding='same', name='block1_pool'))
        model.add(Dropout(0.1, name='block1_dropout'))

        model.add(Conv1D(filters=filter_num[2], kernel_size=kernel_size[2],
                         strides=conv_stride_size[2], padding='same',
                         name='block2_conv1'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='block2_act1'))

        model.add(Conv1D(filters=filter_num[2], kernel_size=kernel_size[2],
                         strides=conv_stride_size[2], padding='same',
                         name='block2_conv2'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='block2_act2'))
        model.add(MaxPooling1D(pool_size=pool_size[2], strides=pool_stride_size[3],
                               padding='same', name='block2_pool'))
        model.add(Dropout(0.1, name='block2_dropout'))

        model.add(Conv1D(filters=filter_num[3], kernel_size=kernel_size[3],
                         strides=conv_stride_size[3], padding='same',
                         name='block3_conv1'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='block3_act1'))
        model.add(Conv1D(filters=filter_num[3], kernel_size=kernel_size[3],
                         strides=conv_stride_size[3], padding='same',
                         name='block3_conv2'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='block3_act2'))
        model.add(MaxPooling1D(pool_size=pool_size[3], strides=pool_stride_size[3],
                               padding='same', name='block3_pool'))
        model.add(Dropout(0.1, name='block3_dropout'))

        model.add(Conv1D(filters=filter_num[4], kernel_size=kernel_size[4],
                         strides=conv_stride_size[4], padding='same',
                         name='block4_conv1'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='block4_act1'))
        model.add(Conv1D(filters=filter_num[4], kernel_size=kernel_size[4],
                         strides=conv_stride_size[4], padding='same',
                         name='block4_conv2'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='block4_act2'))
        model.add(MaxPooling1D(pool_size=pool_size[4], strides=pool_stride_size[4],
                               padding='same', name='block4_pool'))
        model.add(Dropout(0.1, name='block4_dropout'))

        model.add(Flatten(name='flatten'))
        model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name='fc1'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='fc1_act'))

        model.add(Dropout(0.7, name='fc1_dropout'))

        model.add(Dense(512, kernel_initializer=glorot_uniform(seed=0), name='fc2'))
        model.add(BatchNormalization())
        model.add(Activation('relu', name='fc2_act'))

        model.add(Dropout(0.5, name='fc2_dropout'))

        model.add(Dense(classes, kernel_initializer=glorot_uniform(seed=0), name='fc3'))
        model.add(Activation('softmax', name="softmax"))
        return model


In [10]:
model = DFNet.build(input_shape=(5000,1), classes=95)

In [11]:
from keras.optimizers import Adamax

model.compile(loss="categorical_crossentropy", 
              optimizer=Adamax(learning_rate=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0), 
              metrics=["accuracy"])

In [12]:
model.fit(train_data_aug.astype('float32'), to_categorical(train_index_aug), epochs = 30, validation_split=0.3)

Epoch 1/30
998/998 [==============================] - 409s 407ms/step - loss: 3.3917 - accuracy: 0.1557 - val_loss: 2.1204 - val_accuracy: 0.3904
Epoch 2/30
998/998 [==============================] - 386s 387ms/step - loss: 2.1202 - accuracy: 0.4004 - val_loss: 1.3017 - val_accuracy: 0.6288
Epoch 3/30
998/998 [==============================] - 386s 387ms/step - loss: 1.6238 - accuracy: 0.5386 - val_loss: 1.0453 - val_accuracy: 0.7071
Epoch 4/30
998/998 [==============================] - 387s 388ms/step - loss: 1.3325 - accuracy: 0.6204 - val_loss: 0.9099 - val_accuracy: 0.7371
Epoch 5/30
998/998 [==============================] - 437s 438ms/step - loss: 1.1505 - accuracy: 0.6743 - val_loss: 0.7837 - val_accuracy: 0.7722
Epoch 6/30
998/998 [==============================] - 421s 421ms/step - loss: 1.0287 - accuracy: 0.7097 - val_loss: 0.6902 - val_accuracy: 0.7980
Epoch 7/30
998/998 [==============================] - 428s 429ms/step - loss: 0.9253 - accuracy: 0.7378 - val_loss: 0.6380 -

In [13]:
model.evaluate(test_data, to_categorical(test_index))

297/297 [==============================] - 18s 60ms/step - loss: 0.2443 - accuracy: 0.9484


[0.24428589642047882, 0.948421061038971]

In [14]:
model.save('data/model_tor_random_aug.h5')